In [1]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [2]:
from tqdm import tqdm
 

In [3]:
from datasets import load_dataset

dataset = load_dataset("graycatHCO3/CodeAlpaca-20K-Python")


Using the latest cached version of the dataset since graycatHCO3/CodeAlpaca-20K-Python couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\XFZ\.cache\huggingface\datasets\graycatHCO3___code_alpaca-20_k-python\default\0.0.0\757f71b1aefcf442c543ac3496cdf7645b96aef5 (last modified on Tue Apr 30 18:30:43 2024).


# 1. CodeBERT

In [4]:
from transformers import AutoTokenizer

codeRoBerta_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

def tokenize_function(examples):

    inputs = codeRoBerta_tokenizer(
        text=examples["completion"],
        #text_target=examples["prompt"],
        padding="max_length", 
        truncation=True, 
        max_length=512,
        return_tensors="pt"
        )
    
    outputs = codeRoBerta_tokenizer(
        text=examples["prompt"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    return {
        "input_ids":inputs["input_ids"],
        "attention_mask":inputs["attention_mask"],
        "decoder_input_ids":outputs["input_ids"],
        "labels" : outputs["input_ids"].clone(),
    }

cdrba_tkd = dataset.map(tokenize_function, batched=True)

In [5]:
print("List of special tokens:", codeRoBerta_tokenizer.special_tokens_map)

List of special tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}


In [7]:
from transformers import EncoderDecoderModel, RobertaConfig, RobertaModel
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig


cdrba_encoder_config = RobertaConfig.from_pretrained("microsoft/codebert-base")
bartCG_decoder_config = BartConfig.from_pretrained("facebook/bart-large",
                                                     is_decoder=True,
                                                     add_cross_attention=True,
                                                     )




#cdrba_decoder_config.decoder_start_token_id = codeRoBerta_tokenizer.bos_token


#print("Decoder Start Token ID:", cdrba_decoder_config.decoder_start_token_id)


cdrba_encoder = RobertaModel(cdrba_encoder_config).to(device)
bartCG_decoder = BartForConditionalGeneration(bartCG_decoder_config).to(device)

codeRoBerta = EncoderDecoderModel(encoder=cdrba_encoder, decoder=bartCG_decoder).to(device)

codeRoBerta.config.decoder_start_token_id = codeRoBerta_tokenizer.bos_token_id
codeRoBerta.config.pad_token_id = codeRoBerta_tokenizer.pad_token_id



config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

ValueError: The selected decoder is not prepared for the encoder hidden states to be passed. Please see the following discussion on GitHub: https://github.com/huggingface/transformers/issues/23350

In [7]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=codeRoBerta,
    args=training_args,
    train_dataset=cdrba_tkd["train"],
    eval_dataset=cdrba_tkd["test"],
    tokenizer=codeRoBerta_tokenizer,
)



    

In [8]:
torch.__version__

'1.13.0+cu116'

In [9]:
trainer.train()

  0%|          | 0/3585 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [ ]:
del trainer

In [30]:
import transformers
transformers.logging.set_verbosity_debug()


In [31]:
transformers.logging.set_verbosity_info()

# 2. BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration